![alt text](doc/lanChian向量数据库-20241105183941.png)

在上图RAG过程中，向量数据库位于中心位置，其作用的通过用户输入去检索向量数据库，找到和向量数据库语义相近的 Chunk

构建向量数据库需要两个步骤：
- 定义嵌入模型：将 chunk 编码为向量
- 构建向量库

## 什么是嵌入模型

嵌入模型的作用是将输入“文本”转为数字表示的模型，因为计算机能计算的是数字而不是文本

In [1]:
# 加载embedding模型
from langchain.embeddings.ollama import OllamaEmbeddings

embeddings_model=OllamaEmbeddings(model='quentinz/bge-large-zh-v1.5:latest')

C:\Users\wushaogui\AppData\Local\Temp\ipykernel_28920\1405274082.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model=OllamaEmbeddings(model='quentinz/bge-large-zh-v1.5:latest')


LangChain 中的基 Embeddings 类提供了两种方法：一种用于嵌入文档，另一种用于嵌入查询。前者 `.embed_documents`，接受多个文本作为输入，而后者 `.embed_query`，接受单个文本作为输入

In [9]:
texts=[
    "这款笔记本电脑性能卓越，轻薄便携，非常适合商务人士和学生使用。它的续航能力强，屏幕显示清晰，键盘手感舒适，是我近期使用过的最佳便携设备",
    "这款显卡性能强劲，散热良好，非常适合游戏爱好者和专业设计师。它支持最新的图形技术，能够流畅运行各种高负载应用，大幅提升了我的工作效率和游戏体验。",
    "这套鼠标键盘组合设计时尚，响应迅速，使用起来非常舒适。鼠标的DPI可调，适合精确操作；键盘敲击感良好，长时间打字也不会感到疲劳，是办公和游戏的理想选择。"
]

embeddings = embeddings_model.embed_documents(texts)
len(embeddings), len(embeddings[0]),embeddings[0][:5]

(3,
 1024,
 [-0.35345086455345154,
  -0.43917977809906006,
  -0.06953413784503937,
  -0.19146622717380524,
  -0.029116110876202583])

以上代码将每段文本编码为一个长度为1024的向量，通过这个向量，可以比较文本的相似度

In [11]:
# 获取3个文本向量
sentence1_chinese = "我喜欢狗" 
sentence2_chinese = "我喜欢犬科动物" 
sentence3_chinese = "外面的天气很糟糕" 
embedding1_chinese = embeddings_model.embed_query(sentence1_chinese)
embedding2_chinese = embeddings_model.embed_query(sentence2_chinese)
embedding3_chinese = embeddings_model.embed_query(sentence3_chinese)

import numpy as np 
print(np.dot(embedding1_chinese, embedding2_chinese),
np.dot(embedding1_chinese, embedding3_chinese),
np.dot(embedding2_chinese, embedding3_chinese))

377.84199974197213 201.93827333781803 180.74246083979466


**注意**：这里的embedding和词向量不是一个概念，或者说最终的embedding是由多个词向量计算得来，在向量数据库Chroma里面有一段这样的代码：

![alt text](doc/LLMs的Embeddings-20241201114822.png)

**编码得到句子向量的过程**

1. 代码接受2个document输入，每个ducument被padding到256长度
2. 编码每个字符，得到一个384长度的编码，即(2,256,384)
3. 通过加法、标准化得到最终的句子向量(2,384)

其实也可以用神经网络来代替`加法`的过程

- 在传统的序列到序列（seq2seq）模型中，编码器（Encoder）会将不定长的输入文本编码为一个固定长度的向量，这个过程通常通过循环神经网络（RNN），如长短期记忆网络（LSTM）或门控循环单元（GRU）来实现。这个固定长度的向量通常是一个隐藏状态（hidden state）
- 在更现代的模型，如基于Transformer的模型中，每个词或标记（token）都会被编码为一个固定长度的向量，最终取所有token的向量的平均去分类，这个平均向量可以认为是句子向量，还可以像vit一张，新增一个特殊的token，代表这个句子，这个token的向量就是句子向量

## 向量数据库

向量数据库是存储文档嵌入的数据库，并且提供关键词、相似度查询功能

In [4]:
from langchain.vectorstores import Chroma

# 文本分隔 documents -> chunks
from langchain.text_splitter  import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=100,chunk_overlap=20)
documents=text_splitter.create_documents(texts)
split_documents=text_splitter.split_documents(documents)

persist_directory="VectorStore" # 数据库保存路径
db = Chroma.from_documents(split_documents, embeddings_model, persist_directory=persist_directory)
db.persist()

C:\Users\wushaogui\AppData\Local\Temp\ipykernel_28920\1148642834.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


使用similarity_search 方法快速搜索向量数据库中的相似文本片段，约相似的文本排在约前面

In [8]:
query = "我准备买一台个无线鼠标"
docs = db.similarity_search(query)
docs

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={}, page_content='这套鼠标键盘组合设计时尚，响应迅速，使用起来非常舒适。鼠标的DPI可调，适合精确操作；键盘敲击感良好，长时间打字也不会感到疲劳，是办公和游戏的理想选择。'),
 Document(metadata={}, page_content='这款笔记本电脑性能卓越，轻薄便携，非常适合商务人士和学生使用。它的续航能力强，屏幕显示清晰，键盘手感舒适，是我近期使用过的最佳便携设备'),
 Document(metadata={}, page_content='这款显卡性能强劲，散热良好，非常适合游戏爱好者和专业设计师。它支持最新的图形技术，能够流畅运行各种高负载应用，大幅提升了我的工作效率和游戏体验。')]